In [28]:
import site
site.getsitepackages()

['c:\\Users\\Francisco Azeredo\\.conda\\envs\\tese',
 'c:\\Users\\Francisco Azeredo\\.conda\\envs\\tese\\Lib\\site-packages']

In [29]:
import weaviate

In [30]:
client = weaviate.connect_to_local()

In [31]:
response = client.collections.list_all(simple=False)

print(response)

{'DemoCollection': _CollectionConfig(name='DemoCollection', description=None, generative_config=_GenerativeConfig(generative=<GenerativeSearches.OLLAMA: 'generative-ollama'>, model={'model': 'llama3.2'}), inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='page', description=None, data_type=<DataType.INT: 'int'>, index_filterable=True, index_range_filters=False, index_searchable=False, nested_properties=None, tokenization=None, vectorizer_config=None, vectorizer='none'), _Property(name='text', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested

Collection creation

In [32]:
from weaviate.classes.config import Property, DataType, ReferenceProperty, Configure

# Define Ollama API details
OLLAMA_API_ENDPOINT = "http://host.docker.internal:11434"
VECTOR_MODEL = "mxbai-embed-large"
GENERATION_MODEL = "llama3.2"

# Delete old collections if they exist
for collection_name in ["Fluxo", "Etapa", "Entidade", "Pasta", "Ficheiro", "Metadados"]:
    try:
        client.collections.delete(collection_name)
    except:
        pass  # Ignore if collection doesn't exist

# Function to create a collection with vectorization & generative AI
def create_collection(name, description, properties):
    client.collections.create(
        name=name,
        description=description,
        properties=properties,
        vectorizer_config=[
            Configure.NamedVectors.text2vec_ollama(
                name="title_vector",
                source_properties=["name"],  # Change to relevant properties
                api_endpoint=OLLAMA_API_ENDPOINT,
                model=VECTOR_MODEL
            )
        ],
        generative_config=Configure.Generative.ollama(
            api_endpoint=OLLAMA_API_ENDPOINT,
            model=GENERATION_MODEL
        )
    )

# 1. Create "Fluxo" Collection
create_collection(
    "Fluxo",
    "A workflow that contains multiple stages (etapas).",
    [Property(name="name", data_type=DataType.TEXT)]
)

# 2. Create "Etapa" Collection
create_collection(
    "Etapa",
    "A stage within a workflow.",
    [Property(name="name", data_type=DataType.TEXT)]
)

# 3. Create "Entidade" Collection
create_collection(
    "Entidade",
    "An entity that owns folders (pastas).",
    [Property(name="name", data_type=DataType.TEXT)]
)

# 4. Create "Pasta" Collection
create_collection(
    "Pasta",
    "A folder belonging to an entity and containing documents (ficheiros).",
    [Property(name="name", data_type=DataType.TEXT)]
)

# 5. Create "Ficheiro" Collection
create_collection(
    "Ficheiro",
    "A document that contains metadata.",
    [Property(name="name", data_type=DataType.TEXT)]
)

# 6. Create "Metadados" Collection
create_collection(
    "Metadados",
    "Metadata associated with a document (ficheiro).",
    [Property(name="name", data_type=DataType.TEXT)]
)

print("Collections with vectorization & generation created successfully!")

# Create reference properties
# Get collection objects
fluxo = client.collections.get("Fluxo")
etapa = client.collections.get("Etapa")
entidade = client.collections.get("Entidade")
pasta = client.collections.get("Pasta")
ficheiro = client.collections.get("Ficheiro")
metadados = client.collections.get("Metadados")

# Add cross-references
fluxo.config.add_reference(ReferenceProperty(name="hasEtapas", target_collection="Etapa"))
fluxo.config.add_reference(ReferenceProperty(name="belongsToFicheiros", target_collection="Ficheiro"))
fluxo.config.add_reference(ReferenceProperty(name="belongsToPastas", target_collection="Pasta"))

etapa.config.add_reference( ReferenceProperty(name="belongsToFluxo", target_collection="Fluxo"))
etapa.config.add_reference( ReferenceProperty(name="hasFicheiros", target_collection="Ficheiro"))

entidade.config.add_reference( ReferenceProperty(name="hasFicheiros", target_collection="Ficheiro"))
entidade.config.add_reference( ReferenceProperty(name="hasPastas", target_collection="Pasta"))

pasta.config.add_reference( ReferenceProperty(name="hasFicheiros", target_collection="Ficheiro"))
pasta.config.add_reference( ReferenceProperty(name="hasEntidades", target_collection="Entidade"))
pasta.config.add_reference( ReferenceProperty(name="hasFluxos", target_collection="Fluxo"))

ficheiro.config.add_reference( ReferenceProperty(name="belongsToMetadados", target_collection="Metadados"))
ficheiro.config.add_reference( ReferenceProperty(name="hasEtapas", target_collection="Etapa"))
ficheiro.config.add_reference( ReferenceProperty(name="hasPastas", target_collection="Pasta"))
ficheiro.config.add_reference( ReferenceProperty(name="hasEntidades", target_collection="Entidade"))


metadados.config.add_reference( ReferenceProperty(name="hasFicheiros", target_collection="Ficheiro"))
metadados.config.add_reference( ReferenceProperty(name="hasEtapas", target_collection="Etapa"))
metadados.config.add_reference( ReferenceProperty(name="hasPastas", target_collection="Pasta"))
metadados.config.add_reference( ReferenceProperty(name="hasEntidades", target_collection="Entidade"))
print("References added successfully!")



Collections with vectorization & generation created successfully!
References added successfully!


In [33]:
def add_fluxo(fluxo_name,fluxo ,pasta_obj=None, ficheiro_obj=None, etapa_obj=None):
    """Creates a Fluxo and links it to Pasta and Ficheiro if provided."""
    fluxo_obj = fluxo.data.insert({"name": fluxo_name})
    if pasta_obj:
        fluxo.data.reference_add(fluxo_obj, "belongsToPastas", pasta_obj)
    if ficheiro_obj:
        fluxo.data.reference_add(fluxo_obj, "belongsToFicheiros", ficheiro_obj)
    if etapa_obj:
        fluxo.data.reference_add(fluxo_obj, "hasEtapas", etapa_obj)
    return fluxo_obj

def add_etapa(etapa_name, etapa, fluxo_obj=None, ficheiro_obj=None):
    etapa_obj = etapa.data.insert({"name": etapa_name})
    if fluxo_obj:
        fluxo.data.reference_add(fluxo_obj, "hasEtapas", etapa_obj)
        etapa.data.reference_add(etapa_obj, "belongsToFluxo", fluxo_obj)
    if ficheiro_obj:
        etapa.data.reference_add(etapa_obj, "hasFicheiros", ficheiro_obj)
    return etapa_obj

def add_entidade(entidade_name,entidade, ficheiro_obj=None, pasta_obj=None):
    entidade_obj = entidade.data.insert({"name": entidade_name})
    if ficheiro_obj:
        entidade.data.reference_add(entidade_obj, "hasFicheiros", ficheiro_obj)
    if pasta_obj:
        entidade.data.reference_add(entidade_obj, "hasPastas", pasta_obj)
    return entidade_obj

def add_pasta(pasta_name, pasta, entidade_obj=None, fluxo_obj=None, ficheiro_obj=None):
    pasta_obj = pasta.data.insert({"name": pasta_name})
    if entidade_obj:
        entidade.data.reference_add(entidade_obj, "hasPastas", pasta_obj)
        pasta.data.reference_add(pasta_obj, "hasEntidades", entidade_obj)
    if fluxo_obj:
        pasta.data.reference_add(pasta_obj, "hasFluxos", fluxo_obj)
    if ficheiro_obj:
        pasta.data.reference_add(pasta_obj, "hasFicheiros", ficheiro_obj)
    return pasta_obj

def add_ficheiro(ficheiro_name, ficheiro, pasta_obj=None, entidade_obj=None, etapa_obj=None, metadados_obj=None):
    ficheiro_obj = ficheiro.data.insert({"nome": ficheiro_name})
    if pasta_obj:
        pasta.data.reference_add(pasta_obj, "hasFicheiros", ficheiro_obj)
    if entidade_obj:
        entidade.data.reference_add(entidade_obj, "hasFicheiros", ficheiro_obj)
    if etapa_obj:
        etapa.data.reference_add(etapa_obj, "hasFicheiros", ficheiro_obj)
    if metadados_obj:
        ficheiro.data.reference_add(ficheiro_obj, "belongsToMetadados", metadados_obj)
    return ficheiro_obj

def add_metadados(metadados_data, metadados, ficheiro_obj=None, etapa_obj=None, pasta_obj=None, entidade_obj=None):
    metadados_obj = metadados.data.insert({"dados": metadados_data})
    if ficheiro_obj:
        metadados.data.reference_add(metadados_obj, "hasFicheiros", ficheiro_obj)
    if etapa_obj:
        metadados.data.reference_add(metadados_obj, "hasEtapas", etapa_obj)
    if pasta_obj:
        metadados.data.reference_add(metadados_obj, "hasPastas", pasta_obj)
    if entidade_obj:
        metadados.data.reference_add(metadados_obj, "hasEntidades", entidade_obj)
    return metadados_obj

# Configuration for data size
num_entidades = 2
num_pastas_per_entidade = 5
num_ficheiros_per_pasta = 8
num_metadados_per_ficheiro = 1
num_fluxos = 5
num_etapas_per_fluxo = 10

# Step 1: Create Entidades
entidade = client.collections.get("Entidade")
entidade_objs = [add_entidade(f"Empresa {i+1}",entidade) for i in range(num_entidades)]

# Step 2: Create Pastas and link them to Entidades
pasta_objs = []
pasta = client.collections.get("Pasta")
for i, entidade_obj in enumerate(entidade_objs):
    for j in range(num_pastas_per_entidade):
        pasta_obj = add_pasta(f"Pasta {j+1} of Empresa {i+1}",pasta , entidade_obj=entidade_obj)
        pasta_objs.append(pasta_obj)

# Step 3: Create Ficheiros and link them to Pastas and Entidades
ficheiro_objs = []
ficheiro = client.collections.get("Ficheiro")
for i, pasta_obj in enumerate(pasta_objs):
    entidade_obj = entidade_objs[i % len(entidade_objs)]
    for j in range(num_ficheiros_per_pasta):
        ficheiro_obj = add_ficheiro(f"Documento {j+1} in {pasta_obj}",ficheiro, pasta_obj=pasta_obj, entidade_obj=entidade_obj)
        ficheiro_objs.append(ficheiro_obj)

# Step 4: Create Metadados and link them to Ficheiros
metadados_objs = []
metadados = client.collections.get("Metadados")
for i, ficheiro_obj in enumerate(ficheiro_objs):
    for j in range(num_metadados_per_ficheiro):
        metadados_obj = add_metadados(f"Metadata {j+1} for {ficheiro_obj}",metadados,ficheiro_obj=ficheiro_obj)
        metadados_objs.append(metadados_obj)

# Step 5: Create Fluxos and link them to Pastas and Ficheiros
fluxo_objs = []
fluxo = client.collections.get("Fluxo")
for i in range(num_fluxos):
    pasta_obj = pasta_objs[i % len(pasta_objs)]
    ficheiro_obj = ficheiro_objs[i % len(ficheiro_objs)]
    fluxo_obj = add_fluxo(f"Fluxo {i+1}",fluxo, pasta_obj=pasta_obj, ficheiro_obj=ficheiro_obj)
    fluxo_objs.append(fluxo_obj)

# Step 6: Create Etapas and link them to Fluxos and Ficheiros
etapa = client.collections.get("Etapa")
etapa_objs = []
for i, fluxo_obj in enumerate(fluxo_objs):
    ficheiro_obj = ficheiro_objs[i % len(ficheiro_objs)]
    for j in range(num_etapas_per_fluxo):
        etapa_obj = add_etapa(f"Etapa {j+1} of Fluxo {i+1}", etapa,fluxo_obj=fluxo_obj, ficheiro_obj=ficheiro_obj)
        etapa_objs.append(etapa_obj)

print("Bulk sample data inserted and linked successfully!")

Bulk sample data inserted and linked successfully!


Querying Cross Referenced data

In [34]:
#Retrieve "Etapas" from a given "Fluxo"
from weaviate.classes.query import QueryReference

query_result = fluxo.query.fetch_objects(
    return_properties=["name"],
    return_references=QueryReference(
        link_on="hasEtapas",
        return_properties=["name"]
    ),
    limit=10
)
for obj in query_result.objects:
    print(f"Fluxo: {obj.properties['name']}")
    print(f"obj.references: {obj.references.keys()}")
    if "hasEtapas" in obj.references:
        for etapa in obj.references["hasEtapas"].objects:
            print(f"  - Etapa: {etapa.properties['name']}")
    else:
        print("No 'hasEtapas' reference found for this Fluxo.")


Fluxo: Fluxo 1
obj.references: dict_keys(['hasEtapas'])
  - Etapa: Etapa 1 of Fluxo 1
  - Etapa: Etapa 2 of Fluxo 1
  - Etapa: Etapa 3 of Fluxo 1
  - Etapa: Etapa 4 of Fluxo 1
  - Etapa: Etapa 5 of Fluxo 1
  - Etapa: Etapa 6 of Fluxo 1
  - Etapa: Etapa 7 of Fluxo 1
  - Etapa: Etapa 8 of Fluxo 1
  - Etapa: Etapa 9 of Fluxo 1
  - Etapa: Etapa 10 of Fluxo 1
Fluxo: Fluxo 2
obj.references: dict_keys(['hasEtapas'])
  - Etapa: Etapa 1 of Fluxo 2
  - Etapa: Etapa 2 of Fluxo 2
  - Etapa: Etapa 3 of Fluxo 2
  - Etapa: Etapa 4 of Fluxo 2
  - Etapa: Etapa 5 of Fluxo 2
  - Etapa: Etapa 6 of Fluxo 2
  - Etapa: Etapa 7 of Fluxo 2
  - Etapa: Etapa 8 of Fluxo 2
  - Etapa: Etapa 9 of Fluxo 2
  - Etapa: Etapa 10 of Fluxo 2
Fluxo: Fluxo 3
obj.references: dict_keys(['hasEtapas'])
  - Etapa: Etapa 1 of Fluxo 3
  - Etapa: Etapa 2 of Fluxo 3
  - Etapa: Etapa 3 of Fluxo 3
  - Etapa: Etapa 4 of Fluxo 3
  - Etapa: Etapa 5 of Fluxo 3
  - Etapa: Etapa 6 of Fluxo 3
  - Etapa: Etapa 7 of Fluxo 3
  - Etapa: Etapa 8 o

In [35]:
query_result = entidade.query.fetch_objects(
    return_properties=["name"],
    return_references=QueryReference(
        link_on="hasPastas",
        return_properties=["name"],
        return_references=QueryReference(
            link_on="hasFicheiros",
            return_properties=["nome"],
            return_references=QueryReference(
                link_on="belongsToMetadados",
                return_properties=["dados"]
            )
        )
    ),
    limit=10
)
for obj in query_result.objects:
    print(f"Entidade: {obj.properties['name']}")
    for pasta in obj.references["hasPastas"].objects:
        count_ficheiros = 0
        print(f"  - Pasta: {pasta.properties['name'][0:7]}", end="|")
        print(f"- Ficheiros:", {count_ficheiros})
        for ficheiro in pasta.references["hasFicheiros"].objects:
            count_ficheiros += 1
            if "belongsToMetadados" in ficheiro.references:
                for meta in ficheiro.references["belongsToMetadados"].objects:
                    print(f"      - Metadados: {meta.properties['dados']}")
            # else:
            #     print("      - No 'belongsToMetadados' reference found for this Documento.")


Entidade: Empresa 2
  - Pasta: Pasta 1|- Ficheiros: {0}
  - Pasta: Pasta 2|- Ficheiros: {0}
  - Pasta: Pasta 3|- Ficheiros: {0}
  - Pasta: Pasta 4|- Ficheiros: {0}
  - Pasta: Pasta 5|- Ficheiros: {0}
Entidade: Empresa 1
  - Pasta: Pasta 1|- Ficheiros: {0}
  - Pasta: Pasta 2|- Ficheiros: {0}
  - Pasta: Pasta 3|- Ficheiros: {0}
  - Pasta: Pasta 4|- Ficheiros: {0}
  - Pasta: Pasta 5|- Ficheiros: {0}


In [36]:
# Global Semantic Search
from weaviate.classes.query import MetadataQuery
query_text = "Find documents about contract approvals"

# Search across multiple collections
results = []
for collection_name in ["Pasta", "Fluxo", "Etapa", "Metadados", "Ficheiro", "Entidade"]:
    collection = client.collections.get(collection_name)

    search_results = collection.query.hybrid(
        query=query_text,  
        alpha=0.2,  # Set to 0.2 for balanced results
        return_properties=["name"],  # Return these properties
        return_metadata=MetadataQuery(score=True),  # Return relevance score
        limit=5,  # Get top 5 results per collection
    )

    # Store results with class name
    for obj in search_results.objects:
        results.append({
            "class": collection_name,
            "name": obj.properties["name"],
            "score": obj.metadata.score,
        })

# Sort results by relevance score (higher is better)
results.sort(key=lambda x: x["score"], reverse=True)

# Display results
for result in results:
    print(f"[{result['class']}] {result['name']} (Score: {result['score']})")


[Pasta] Pasta 2 of Empresa 1 (Score: 0.20000000298023224)
[Fluxo] Fluxo 3 (Score: 0.20000000298023224)
[Etapa] Etapa 8 of Fluxo 1 (Score: 0.20000000298023224)
[Metadados] None (Score: 0.20000000298023224)
[Metadados] None (Score: 0.20000000298023224)
[Metadados] None (Score: 0.20000000298023224)
[Metadados] None (Score: 0.20000000298023224)
[Metadados] None (Score: 0.20000000298023224)
[Ficheiro] None (Score: 0.20000000298023224)
[Ficheiro] None (Score: 0.20000000298023224)
[Ficheiro] None (Score: 0.20000000298023224)
[Ficheiro] None (Score: 0.20000000298023224)
[Ficheiro] None (Score: 0.20000000298023224)
[Entidade] Empresa 2 (Score: 0.20000000298023224)
[Etapa] Etapa 9 of Fluxo 1 (Score: 0.19135645031929016)
[Etapa] Etapa 7 of Fluxo 1 (Score: 0.18554548919200897)
[Fluxo] Fluxo 2 (Score: 0.18322160840034485)
[Etapa] Etapa 9 of Fluxo 4 (Score: 0.18066634237766266)
[Etapa] Etapa 9 of Fluxo 2 (Score: 0.17712409794330597)
[Pasta] Pasta 2 of Empresa 2 (Score: 0.14277087152004242)
[Pasta] P

In [37]:
client.close()